In [1]:
import numpy
import pandas as pd
import bs4
import spider_runner
from morecomplicatedsentiment.spiders import ispytanogetlinks, ispytanogetreviews
import os
import warnings
warnings.filterwarnings('ignore')

## Загрузка выборок

В тестовой выборке находятся отзывы о мобильных телефонах. Решено брать отзывы с сайта https://ispytano.ru

Парсить будем с помощью фреймворка scrapy.

In [2]:
%%time
if os.path.exists('ispytano_links.csv'):
    os.remove('ispytano_links.csv')
if os.path.exists('ispytano_reviews.csv'):
    os.remove('ispytano_reviews.csv')
spider_runner.run_spider(ispytanogetlinks.IspytanogetlinksSpider, 
                         ['https://ispytano.ru/-reiting_cat_121'],
                         settings={
                             'FEED_FORMAT': 'csv',
                             'FEED_URI': 'ispytano_links.csv'
                         })
urls = pd.read_csv('ispytano_links.csv')
spider_runner.run_spider(ispytanogetreviews.IspytanogetreviewsSpider, 
                         list(urls.link),
                         settings={
                             'FEED_FORMAT': 'csv',
                             'FEED_URI': 'ispytano_reviews.csv'
                         })

Wall time: 57.2 s


In [2]:
reviews = pd.read_csv('ispytano_reviews.csv')

In [3]:
reviews.head()

,labels,reviews
0,darkblue,- Очень мало внутренней памяти (это один из са...
1,red,Низкая чувствительность экрана. Стилусы для iP...
2,darkgreen,перегрев при работе и одновременной зарядке в ...
3,darkgreen,- При активном использовании аккумулятора хват...
4,darkgreen,"1. Очень малый объем встроенной памяти, что о ..."


Отзывы имеют три градации:

* положительные (darkgreen)
* нейтральные (darkblue)
* отрицательные (red)

Всего отзывов в обучающей выбоке:

In [4]:
reviews.shape

(17884, 2)

Из них нейтральных отзывов:

In [5]:
reviews[reviews.labels == 'darkblue'].shape

(1770, 2)

Удалим нейтральные отзывы.

In [6]:
reviews = reviews[reviews.labels != 'darkblue']

Видим, что выборка не сильно уменьшилась.

In [7]:
reviews.shape

(16114, 2)

Поправим оценки отзывов:

* 0 - отрицательные отзывы
* 1 - положительные отзывы

In [8]:
reviews.labels = reviews.labels.apply(lambda x: 0 if x == 'red' else 1)

In [9]:
reviews.head()

,labels,reviews
1,0,Низкая чувствительность экрана. Стилусы для iP...
2,1,перегрев при работе и одновременной зарядке в ...
3,1,- При активном использовании аккумулятора хват...
4,1,"1. Очень малый объем встроенной памяти, что о ..."
5,0,"Нет передачи музыки через блюпуп, аккумулятор ..."


Извлечем тестовую выборку.

In [10]:
test_parser = bs4.BeautifulSoup(open('test.csv', encoding='utf8'), 'lxml')

In [11]:
test_reviews = pd.DataFrame()
test_reviews['reviews'] = [test_review.text for test_review in test_parser.find_all('review')]

In [12]:
test_reviews.head()

,reviews
0,"Ужасно слабый аккумулятор, это основной минус ..."
1,ценанадежность-неубиваемостьдолго держит батар...
2,"подробнее в комментариях\nК сожалению, факт по..."
3,я любительница громкой музыки. Тише телефона у...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех..."


In [13]:
test_reviews.shape

(100, 1)

## Предобработка

Произведем первоначальную предобработку отзывов. Для этого удалим пунктуацию, стоп-слова и попытаемся привести слова к начальной форме с помощью модуля spaCy.

Инструкция по установке модуля spaCy: https://github.com/buriy/spacy-ru

In [14]:
import spacy
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from string import punctuation

import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\d.v.kulemin.UTMN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Хочется функцию предобработки добавить в будующий пайплайн. Для этого нам понадобится следующая функция.

Код взят от сюда: https://ryan-cranfill.github.io/sentiment-pipeline-sklearn-3/

In [15]:
def pipelinize(function, active=True):
    def list_comprehend_a_function(list_or_series, active=True):
        if active:
            return [function(i) for i in list_or_series]
        else: # if it's not active, just pass it right back
            return list_or_series
    return FunctionTransformer(list_comprehend_a_function, validate=False, kw_args={'active':active})

Код функции предобработки отзывов написан на основе: https://www.kaggle.com/alxmamaev/how-to-easy-preprocess-russian-text

In [16]:
russian_stopwords = nltk.corpus.stopwords.words('russian')

In [17]:
nlp = spacy.load('ru2', disable=["parser", "ner"])

In [18]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if str(token) not in russian_stopwords \
              and str(token) != ' ' \
              and str(token).strip() not in punctuation \
              and not str(token).isdigit()]
    text = ' '.join(tokens)
    
    return text

In [19]:
def lemmatize_texts(df, col):
    lemmatized_texts = []
    for idx, text in zip(df.index, df[col]):
        print(f'Производится лемматизация текста: {idx}')
        lemmatized_texts.append(preprocess_text(text))
    return lemmatized_texts

Для ускорения подбора параметров произведем предобработку отзывов и добавим столбец в обучающую выборку. Подбирать параметры будем на новом столбце.

In [20]:
%%time
reviews['lemmatized_reviews'] = lemmatize_texts(reviews, 'reviews')

Производится лемматизация текста: 1
Производится лемматизация текста: 2
Производится лемматизация текста: 3
Производится лемматизация текста: 4
Производится лемматизация текста: 5
Производится лемматизация текста: 7
Производится лемматизация текста: 8
Производится лемматизация текста: 9
Производится лемматизация текста: 10
Производится лемматизация текста: 11
Производится лемматизация текста: 12
Производится лемматизация текста: 13
Производится лемматизация текста: 14
Производится лемматизация текста: 15
Производится лемматизация текста: 17
Производится лемматизация текста: 18
Производится лемматизация текста: 19
Производится лемматизация текста: 20
Производится лемматизация текста: 21
Производится лемматизация текста: 22
Производится лемматизация текста: 23
Производится лемматизация текста: 24
Производится лемматизация текста: 25
Производится лемматизация текста: 26
Производится лемматизация текста: 27
Производится лемматизация текста: 28
Производится лемматизация текста: 29
Производи

Производится лемматизация текста: 483
Производится лемматизация текста: 484
Производится лемматизация текста: 485
Производится лемматизация текста: 486
Производится лемматизация текста: 487
Производится лемматизация текста: 488
Производится лемматизация текста: 489
Производится лемматизация текста: 490
Производится лемматизация текста: 491
Производится лемматизация текста: 492
Производится лемматизация текста: 493
Производится лемматизация текста: 494
Производится лемматизация текста: 495
Производится лемматизация текста: 496
Производится лемматизация текста: 497
Производится лемматизация текста: 498
Производится лемматизация текста: 499
Производится лемматизация текста: 500
Производится лемматизация текста: 501
Производится лемматизация текста: 502
Производится лемматизация текста: 503
Производится лемматизация текста: 504
Производится лемматизация текста: 505
Производится лемматизация текста: 506
Производится лемматизация текста: 507
Производится лемматизация текста: 508
Производится

Производится лемматизация текста: 969
Производится лемматизация текста: 970
Производится лемматизация текста: 972
Производится лемматизация текста: 973
Производится лемматизация текста: 974
Производится лемматизация текста: 975
Производится лемматизация текста: 976
Производится лемматизация текста: 977
Производится лемматизация текста: 978
Производится лемматизация текста: 979
Производится лемматизация текста: 980
Производится лемматизация текста: 981
Производится лемматизация текста: 982
Производится лемматизация текста: 983
Производится лемматизация текста: 984
Производится лемматизация текста: 985
Производится лемматизация текста: 986
Производится лемматизация текста: 987
Производится лемматизация текста: 988
Производится лемматизация текста: 989
Производится лемматизация текста: 990
Производится лемматизация текста: 991
Производится лемматизация текста: 992
Производится лемматизация текста: 993
Производится лемматизация текста: 995
Производится лемматизация текста: 996
Производится

Производится лемматизация текста: 1452
Производится лемматизация текста: 1453
Производится лемматизация текста: 1454
Производится лемматизация текста: 1455
Производится лемматизация текста: 1457
Производится лемматизация текста: 1458
Производится лемматизация текста: 1459
Производится лемматизация текста: 1460
Производится лемматизация текста: 1462
Производится лемматизация текста: 1463
Производится лемматизация текста: 1464
Производится лемматизация текста: 1465
Производится лемматизация текста: 1466
Производится лемматизация текста: 1468
Производится лемматизация текста: 1469
Производится лемматизация текста: 1470
Производится лемматизация текста: 1472
Производится лемматизация текста: 1473
Производится лемматизация текста: 1474
Производится лемматизация текста: 1477
Производится лемматизация текста: 1478
Производится лемматизация текста: 1479
Производится лемматизация текста: 1480
Производится лемматизация текста: 1482
Производится лемматизация текста: 1484
Производится лемматизация

Производится лемматизация текста: 1930
Производится лемматизация текста: 1931
Производится лемматизация текста: 1932
Производится лемматизация текста: 1933
Производится лемматизация текста: 1934
Производится лемматизация текста: 1935
Производится лемматизация текста: 1936
Производится лемматизация текста: 1937
Производится лемматизация текста: 1938
Производится лемматизация текста: 1939
Производится лемматизация текста: 1940
Производится лемматизация текста: 1941
Производится лемматизация текста: 1942
Производится лемматизация текста: 1943
Производится лемматизация текста: 1944
Производится лемматизация текста: 1946
Производится лемматизация текста: 1947
Производится лемматизация текста: 1948
Производится лемматизация текста: 1949
Производится лемматизация текста: 1950
Производится лемматизация текста: 1952
Производится лемматизация текста: 1953
Производится лемматизация текста: 1954
Производится лемматизация текста: 1955
Производится лемматизация текста: 1956
Производится лемматизация

...

Производится лемматизация текста: 15715
Производится лемматизация текста: 15716
Производится лемматизация текста: 15717
Производится лемматизация текста: 15718
Производится лемматизация текста: 15719
Производится лемматизация текста: 15720
Производится лемматизация текста: 15721
Производится лемматизация текста: 15723
Производится лемматизация текста: 15725
Производится лемматизация текста: 15726
Производится лемматизация текста: 15727
Производится лемматизация текста: 15730
Производится лемматизация текста: 15731
Производится лемматизация текста: 15733
Производится лемматизация текста: 15735
Производится лемматизация текста: 15736
Производится лемматизация текста: 15737
Производится лемматизация текста: 15738
Производится лемматизация текста: 15739
Производится лемматизация текста: 15740
Производится лемматизация текста: 15741
Производится лемматизация текста: 15742
Производится лемматизация текста: 15744
Производится лемматизация текста: 15745
Производится лемматизация текста: 15747


Производится лемматизация текста: 16181
Производится лемматизация текста: 16182
Производится лемматизация текста: 16183
Производится лемматизация текста: 16184
Производится лемматизация текста: 16185
Производится лемматизация текста: 16186
Производится лемматизация текста: 16187
Производится лемматизация текста: 16188
Производится лемматизация текста: 16189
Производится лемматизация текста: 16190
Производится лемматизация текста: 16191
Производится лемматизация текста: 16192
Производится лемматизация текста: 16193
Производится лемматизация текста: 16194
Производится лемматизация текста: 16195
Производится лемматизация текста: 16196
Производится лемматизация текста: 16197
Производится лемматизация текста: 16200
Производится лемматизация текста: 16201
Производится лемматизация текста: 16202
Производится лемматизация текста: 16203
Производится лемматизация текста: 16204
Производится лемматизация текста: 16205
Производится лемматизация текста: 16206
Производится лемматизация текста: 16207


Производится лемматизация текста: 16623
Производится лемматизация текста: 16624
Производится лемматизация текста: 16625
Производится лемматизация текста: 16627
Производится лемматизация текста: 16628
Производится лемматизация текста: 16629
Производится лемматизация текста: 16630
Производится лемматизация текста: 16631
Производится лемматизация текста: 16632
Производится лемматизация текста: 16633
Производится лемматизация текста: 16634
Производится лемматизация текста: 16635
Производится лемматизация текста: 16636
Производится лемматизация текста: 16637
Производится лемматизация текста: 16638
Производится лемматизация текста: 16639
Производится лемматизация текста: 16640
Производится лемматизация текста: 16641
Производится лемматизация текста: 16642
Производится лемматизация текста: 16643
Производится лемматизация текста: 16644
Производится лемматизация текста: 16645
Производится лемматизация текста: 16646
Производится лемматизация текста: 16648
Производится лемматизация текста: 16649


Производится лемматизация текста: 17058
Производится лемматизация текста: 17059
Производится лемматизация текста: 17060
Производится лемматизация текста: 17061
Производится лемматизация текста: 17062
Производится лемматизация текста: 17063
Производится лемматизация текста: 17064
Производится лемматизация текста: 17065
Производится лемматизация текста: 17067
Производится лемматизация текста: 17069
Производится лемматизация текста: 17070
Производится лемматизация текста: 17071
Производится лемматизация текста: 17072
Производится лемматизация текста: 17073
Производится лемматизация текста: 17074
Производится лемматизация текста: 17075
Производится лемматизация текста: 17076
Производится лемматизация текста: 17077
Производится лемматизация текста: 17078
Производится лемматизация текста: 17079
Производится лемматизация текста: 17080
Производится лемматизация текста: 17081
Производится лемматизация текста: 17082
Производится лемматизация текста: 17083
Производится лемматизация текста: 17084


Производится лемматизация текста: 17503
Производится лемматизация текста: 17504
Производится лемматизация текста: 17505
Производится лемматизация текста: 17506
Производится лемматизация текста: 17507
Производится лемматизация текста: 17508
Производится лемматизация текста: 17510
Производится лемматизация текста: 17511
Производится лемматизация текста: 17512
Производится лемматизация текста: 17513
Производится лемматизация текста: 17514
Производится лемматизация текста: 17515
Производится лемматизация текста: 17517
Производится лемматизация текста: 17518
Производится лемматизация текста: 17520
Производится лемматизация текста: 17521
Производится лемматизация текста: 17522
Производится лемматизация текста: 17523
Производится лемматизация текста: 17524
Производится лемматизация текста: 17525
Производится лемматизация текста: 17526
Производится лемматизация текста: 17527
Производится лемматизация текста: 17528
Производится лемматизация текста: 17529
Производится лемматизация текста: 17530


In [22]:
reviews.head()

,labels,reviews,lemmatized_reviews
1,0,Низкая чувствительность экрана. Стилусы для iP...,низкий чувствительность экран стилусы iphone n...
2,1,перегрев при работе и одновременной зарядке в ...,перегрев работе одновременный зарядка жаркий а...
3,1,- При активном использовании аккумулятора хват...,при активный использование аккумулятор хватать...
4,1,"1. Очень малый объем встроенной памяти, что о ...",очень малый объём встроить память снижать функ...
5,0,"Нет передачи музыки через блюпуп, аккумулятор ...",нет передача музыка блюпуп аккумулятор слабый ...


## Подбор модели

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold

In [25]:
def review_classifier(name_method_list):
    return Pipeline([(name, method) for (name, method) in name_method_list])

Быстренько пробежимся по разным вариантам векторизаторов и классификаторов с обучением на предобработанных и не предобработанных текстах.

In [27]:
%%time
scores = []
for rev, r_name in zip([reviews.lemmatized_reviews, reviews.reviews], 
                       ['Lemmatized', 'Nonlemmatized']):
    for vct, v_name in zip([CountVectorizer(), TfidfVectorizer()], 
                           ['CountVectorizer', 'TfidfVectorizer']):
        for clf, c_name in zip([LogisticRegression(solver='liblinear', multi_class='auto', random_state=0), 
                                LinearSVC(random_state=0), 
                                MultinomialNB()], 
                               ['LogisticRegression', 
                                'LinearSVC', 
                                'MultinomialNB']):
            score = cross_val_score(review_classifier([(v_name, vct), (c_name, clf)]), 
                                    rev, 
                                    reviews.labels, cv=10).mean()
            scores.append((f'{r_name} + {v_name} + {c_name}', score))
            print(f'{r_name}, vectorizer: {v_name}, classifier: {c_name}')
            print(score)

Lemmatized, vectorizer: CountVectorizer, classifier: LogisticRegression
0.9231118320669512
Lemmatized, vectorizer: CountVectorizer, classifier: LinearSVC
0.9123141654978962
Lemmatized, vectorizer: CountVectorizer, classifier: MultinomialNB
0.8954314689518055
Lemmatized, vectorizer: TfidfVectorizer, classifier: LogisticRegression
0.9101403295162062
Lemmatized, vectorizer: TfidfVectorizer, classifier: LinearSVC
0.9280141947813758
Lemmatized, vectorizer: TfidfVectorizer, classifier: MultinomialNB
0.862417890664735
Nonlemmatized, vectorizer: CountVectorizer, classifier: LogisticRegression
0.9254081192299985
Nonlemmatized, vectorizer: CountVectorizer, classifier: LinearSVC
0.91386580460213
Nonlemmatized, vectorizer: CountVectorizer, classifier: MultinomialNB
0.8917084598431023
Nonlemmatized, vectorizer: TfidfVectorizer, classifier: LogisticRegression
0.9052377356164172
Nonlemmatized, vectorizer: TfidfVectorizer, classifier: LinearSVC
0.9294415332213368
Nonlemmatized, vectorizer: TfidfVector

In [28]:
scores.sort(key=lambda x: x[1], reverse=True)
scores

[('Nonlemmatized + TfidfVectorizer + LinearSVC', 0.9294415332213368),
 ('Lemmatized + TfidfVectorizer + LinearSVC', 0.9280141947813758),
 ('Nonlemmatized + CountVectorizer + LogisticRegression', 0.9254081192299985),
 ('Lemmatized + CountVectorizer + LogisticRegression', 0.9231118320669512),
 ('Nonlemmatized + CountVectorizer + LinearSVC', 0.91386580460213),
 ('Lemmatized + CountVectorizer + LinearSVC', 0.9123141654978962),
 ('Lemmatized + TfidfVectorizer + LogisticRegression', 0.9101403295162062),
 ('Nonlemmatized + TfidfVectorizer + LogisticRegression', 0.9052377356164172),
 ('Lemmatized + CountVectorizer + MultinomialNB', 0.8954314689518055),
 ('Nonlemmatized + CountVectorizer + MultinomialNB', 0.8917084598431023),
 ('Lemmatized + TfidfVectorizer + MultinomialNB', 0.862417890664735),
 ('Nonlemmatized + TfidfVectorizer + MultinomialNB', 0.862417890664735)]

Видим, что качество на не предобработанных отзывах не сильно отличается от качества на предобработанных. Поэтому будем обучаться на предобработанных отзывах.

In [26]:
def search_by_grid(X, y, model, parameters_grid, cv, scoring='accuracy', verbose=2):
    grid_cv = GridSearchCV(model, parameters_grid, scoring=scoring, cv=cv, verbose=verbose)
    grid_cv.fit(X, y)
    return grid_cv

In [28]:
model = Pipeline([('vectorizer', TfidfVectorizer()), ('classifier', LinearSVC(random_state=0))])
parameters_grid = {
                        'vectorizer__analyzer': ['word', 'char', 'char_wb'],
                        'vectorizer__ngram_range': [(i, j) for i in range(1, 4) for j in range(1, 4) if j >= i],
                        'vectorizer__norm': ['l1', 'l2', None],
                   }

In [32]:
%%time
grid_cv = search_by_grid(reviews.lemmatized_reviews, reviews.labels, model, parameters_grid, StratifiedKFold(n_splits=10))

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1 
[CV]  vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1, total=   2.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s


[CV] vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1 
[CV]  vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1, total=   2.0s
[CV] vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1 
[CV]  vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1, total=   2.0s
[CV] vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1 
[CV]  vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1, total=   2.0s
[CV] vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1 
[CV]  vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1, total=   2.1s
[CV] vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1 
[CV]  vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer__norm=l1, total=   2.0s
[CV] vectorizer__analyzer=word, vectorizer__ngram_range=(1, 1), vectorizer_

...

[CV]  vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None, total=  13.1s
[CV] vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None 
[CV]  vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None, total=   9.5s
[CV] vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None 
[CV]  vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None, total=  13.3s
[CV] vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None 
[CV]  vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None, total=  13.2s
[CV] vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None 
[CV]  vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None, total=  13.7s
[CV] vectorizer__analyzer=char, vectorizer__ngram_range=(3, 3), vectorizer__norm=None 
[CV]  vectorizer__analyzer=char, vectorizer__ngram_rang

[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2, total=  21.5s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2, total=  21.7s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2, total=  21.2s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2, total=  21.3s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=l2, total=  21.3s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(1, 3), vectorizer__norm=None 
[CV]  vectorizer__analyzer=char_wb, vectori

[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2, total=   8.2s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2, total=   8.2s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2, total=   8.2s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2, total=   8.3s
[CV] vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2 
[CV]  vectorizer__analyzer=char_wb, vectorizer__ngram_range=(3, 3), vectorizer__norm=l2, total=   8.2s
[CV] vectorizer__analyzer=char_wb, vectorizer

[Parallel(n_jobs=1)]: Done 540 out of 540 | elapsed: 114.3min finished


Wall time: 1h 54min 17s


In [33]:
grid_cv.best_estimator_

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
   

In [34]:
grid_cv.best_score_

0.9280129080302842

* vectorizer__analyzer = word
* vectorizer__ngram_range = (1, 1)
* vectorizer__norm = l2
* vectorizer__max_df = 1.0
* classifier__C = 1.0

0.9280129080302842

In [30]:
parameters_grid = {
                        'vectorizer__ngram_range': [(1, 1), (1, 3)],
                        'vectorizer__max_df': numpy.arange(.1, 1.1, .1),
                        'classifier__C': numpy.append(numpy.arange(.1, 1.1, .1), [10, 100, 1000])
                   }

In [52]:
%%time
grid_cv = search_by_grid(reviews.lemmatized_reviews, reviews.labels, model, parameters_grid, StratifiedKFold(n_splits=10))

Fitting 10 folds for each of 260 candidates, totalling 2600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   1.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   2.0s
[CV] classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   1.9s
[CV] classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   1.9s
[CV] classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   1.9s
[CV] classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   1.8s
[CV] classifier__C=0.1, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=0.1, vectorizer__max_df=0

...

[CV]  classifier__C=100.0, vectorizer__max_df=1.0, vectorizer__ngram_range=(1, 3), total=  43.1s
[CV] classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   3.2s
[CV] classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   3.4s
[CV] classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   3.6s
[CV] classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   3.5s
[CV] classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.1, vectorizer__ngram_range=(1, 1), total=   3.6

[CV] classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 1), total=   3.7s
[CV] classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 1), total=   3.8s
[CV] classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 1), total=   3.8s
[CV] classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 1) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 1), total=   4.0s
[CV] classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 3), total=  40.7s
[CV] classifier__C=1000.0, vectorizer__max_df=0.5, vectorizer__ngram_range=(1, 3) 
[CV]  classi

[CV]  classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3), total=  48.7s
[CV] classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3), total=  48.5s
[CV] classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3), total=  48.7s
[CV] classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3), total=  48.1s
[CV] classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3), total=  48.3s
[CV] classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=1000.0, vectorizer__max_df=0.9, vectorizer__ngram_range=(1, 3), total=  48.

[Parallel(n_jobs=1)]: Done 2600 out of 2600 | elapsed: 507.2min finished


Wall time: 8h 27min 59s


In [53]:
grid_cv.best_estimator_

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.30000000000000004,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=100.0, class_weight=None, dual=True,
                    

In [54]:
grid_cv.best_score_

0.9309916842497208

* vectorizer__analyzer = word
* vectorizer__ngram_range = (1, 3)
* vectorizer__norm = l2
* vectorizer__max_df = 0.3
* classifier__C = 100

0.9309916842497208

In [35]:
parameters_grid = {
                        'vectorizer__ngram_range': [(1, 3)],
                        'vectorizer__max_df': numpy.arange(.25, 0.35, .01),
                        'classifier__C': [100]
                   }

In [36]:
%%time
grid_cv = search_by_grid(reviews.lemmatized_reviews, reviews.labels, model, parameters_grid, StratifiedKFold(n_splits=10))

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3), total=  51.7s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   51.6s remaining:    0.0s


[CV] classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3), total=  53.1s
[CV] classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3), total=  53.7s
[CV] classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3), total=  55.8s
[CV] classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3), total=  51.8s
[CV] classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3), total=  48.1s
[CV] classifier__C=100, vectorizer__max_df=0.25, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorize

[CV]  classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3), total=  52.9s
[CV] classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3), total=  50.0s
[CV] classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3), total=  50.4s
[CV] classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3), total=  52.0s
[CV] classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3) 
[CV]  classifier__C=100, vectorizer__max_df=0.33000000000000007, vectorizer__ngram_range=(1, 3), total=  53.4s
[CV] classifier__C=100, vectorizer__max_df=0.3400000000000001

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 86.6min finished


Wall time: 1h 27min 35s


In [37]:
grid_cv.best_estimator_

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.31000000000000005,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=100, class_weight=None, dual=True,
                      

In [38]:
grid_cv.best_score_

0.9310537420876257

* vectorizer__analyzer = word
* vectorizer__ngram_range = (1, 3)
* vectorizer__norm = l2
* vectorizer__max_df = 0.31
* classifier__C = 100

0.9310537420876257

## Обучение финальной модели

In [39]:
final_model = Pipeline([
    ('lemmatizer', pipelinize(preprocess_text)), 
    ('vectorizer', TfidfVectorizer(ngram_range=(1,3), max_df=0.31)),
    ('classifier', LinearSVC(C=100))
])

In [40]:
final_model.fit(reviews.reviews, reviews.labels)

Pipeline(memory=None,
         steps=[('lemmatizer',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function pipelinize.<locals>.list_comprehend_a_function at 0x000001511FD1AAE8>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args={'active': True},
                                     pass_y='deprecated', validate=False)),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error...
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=100, class_weight=None,

In [58]:
answer = pd.DataFrame({'Id': test_reviews.index, 
                       'y': ['neg' if p == 0 else 'pos' for p in final_model.predict(test_reviews.reviews)]})
answer.to_csv(f'submition1.csv', index=False)

По состоянию на 04.10.2019:

![title](score.jpg)

In [66]:
from sklearn.svm import SVC

In [71]:
final_model2 = Pipeline([
    ('lemmatizer', pipelinize(preprocess_text)), 
    ('vectorizer', TfidfVectorizer(ngram_range=(1,3), max_df=0.31)),
    ('classifier', SVC(C=100, probability=True, kernel='linear'))
])

In [72]:
%%time
final_model2.fit(reviews.reviews, reviews.labels)

Wall time: 1h 7min 48s


Pipeline(memory=None,
         steps=[('lemmatizer',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function pipelinize.<locals>.list_comprehend_a_function at 0x0000015125BFCAE8>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args={'active': True},
                                     pass_y='deprecated', validate=False)),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error...
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 SVC(C=100, cache_size=200, class_we

In [73]:
pred2 = final_model2.predict(test_reviews.reviews)

In [75]:
probs = final_model2.predict_proba(test_reviews.reviews)

In [80]:
import pickle
from sklearn.externals import joblib

C:\Users\d.v.kulemin.UTMN\AppData\Local\Programs\Python\Virtualenvs\v36_64\venv\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [81]:
joblib.dump(final_model2, 'LemmatizedTfidfLinearSVCTextSentiment.pkl')

['LemmatizedTfidfLinearSVCTextSentiment.pkl']

In [83]:
final_model2.predict(["Самый классный телефон на свете! :)"])

array([1], dtype=int64)

In [65]:
pred = final_model.predict(test_reviews.reviews)

In [63]:
pred

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0], dtype=int64)

In [74]:
pred2

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0], dtype=int64)

In [77]:
pred == pred2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])